# Initialization code for simulating an idealized baroclinic wave in the WRF model

## Components
0. Import modules
1. User options and parameters
2. WRF-specific parameters
3. Initial jet
4. QGPV anomaly
5. Moisture and WRF grid
6. WRF input file

## 0) Import modules

In [ ]:
%matplotlib notebook
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
from wrf import interplevel, getvar, destagger
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from bwave_ideal_wrf import epv_jet, qgpv_pert, wrf_fields, write_wrfinputd01

G = 9.81  # gravitational acceleration, in m/s^2
T0 = 300.  # reference potential temperature, in K
P0 = 1.0e5  # reference pressure, in Pa
CP = 1004.  # specific heat at constant pressure, in J/(K*kg)       
CV = 717.  # specific heat at constant volume, in J/(K*kg)
RD = 287.  # ideal gas constant for dry air, in J/(K*kg)
RV = 461.6 # ideal gas constant for water vapor, in J/(K*kg)
F0 = 1.0e-4  # Coriolis parameter, in s^-1
SVPT0 = 273.15
GAMMA = CP/CV
KAPPA = RD/CP

## 1) User options and parameters

In [ ]:
### Background state options
psurf_gradient = False
shear = False
baroclinic = False

### PV perturbation options
up_pert = True
pert_type = 2 # 1: Gaussian, 2: Cosine, 3: Theta
surf_pert = True
extra_surfpert = False
extra_upperts = False
extra_lowperts = False
external_qg_pert = False

### Grid parameters
nx = 400 #270  # number of grid points in x direction
ny = 360 #240  # number of grid points in y direction
nz = 100  # number of grid points in z direction
hres = 20. #30.  # horizontal grid resolution, in km
zl = 20.  # height of top of atmosphere, in km
pbot = 101200.  # bottom pressure, in Pa
ptop = 5000.  # top pressure, in Pa

### Anomaly grid parameters
x_pert = 150 #100 # upper
y_pert = 180 #120 #110 # upper
#th_pert = 310 # upper
z_pert = 50  # upper (for nz = 100 if you use options 1 or 2)

xs_pert = 208 #140 # surface
ys_pert = 136 #90 # surface

xsp_pert = 120 # extra surface
ysp_pert = 87  # extra surface

x_pert_neg = 133 #100   # extra upper
y_pert_neg = 119  # extra upper
#th_pert_neg = 308
z_pert_neg = 50   #  extra upper

x_pert_pos = 100 #133   # extra upper
y_pert_pos = 119
#th_pert_pos = 310 # extra upper
z_pert_pos = 50   # extra upper 

xlow_pert_neg = 120   # extra lower
ylow_pert_neg = 86  # extra lower
zlow_pert_neg = 18   #  extra lower

xlow_pert_pos = 105 # extra lower
ylow_pert_pos = 80 # extra lower
zlow_pert_pos = 5 # extra lower

### Jet parameters
pibot = 1004. #CP*(pbot/P0)**(RD/CP)  # bottom height, in pi
pitop = 424. #CP*(ptop/P0)**(RD/CP)  # top height, in pi
npi_h = 180  # number of grid points in pi
npi_l = 40  # number of grid points in pi for low-resolution run
ny_l = 30  # number of grid points in y for low-resolution EPV inversion
thtop = 526 # top potential temperature, in K
dth = 10 # max displacement from thtop
dyth = 1e6  # meridional scale for theta transition, in m
pim = 660. # mean height of tropopause, in pi coordinates
dpitr = 30.  # max displacement from pim
dytr = 5e5  # meridional scale for tropopause transition, in m
pvs = 5.0e-6  # PV at bottom of stratosphere, in (m^2*K)/(s*kg)
pvt = 0.3e-6  # PV at bottom of troposhere
dpipv = 15. # tropopause depth in pi over which strongest PV change occurs
nit = 50000  # number of iterations
om = 1.8  # successive over-relaxation coefficient
avg_rh = 0.72 # average relative humidity in the domain for moist soundings
c = 1.0  # additional jet speed if pressure gradient option is enabled
c_shear_linear = 0.0 #-5.75e-7 # s^{-1}, positive = anticyclonic, negative = cyclonic
c_shear_sin = 5.0 # ms^{-1}, positive = anticyclonic, negative = cyclonic
decay_shear = 1.5e6
p_mid = pbot

### Perturbation parameters
qgpv_mag = 1.0e-4  # magnitude of QGPV anomaly, in s-1
theta_mag = 0.  # magnitude of boundary theta, in K
az = 1.5  # vertical decay scale of anomaly, in km
ax = 200. # x decay scale of anomaly, in km
ay = 600. # y decay scale of anomaly, in km
#ath = 2.0 # theta decay scale of anomaly, in K

dtheta_surf = 5.0
dqgpv_surf = 0.
axs = 600.
ayn = 200.
ays = 200.
azs = 1.0

dtheta_pert = 1.0
dqgpv_pert = 0.
axsp = 150.
aynp = 150.
aysp = 150.
azsp = 1.0

qgpv_mag_neg = -4e-5
theta_mag_neg = 0.
az_neg = 2.5
ax_neg = 1000.
ay_neg = 500.
#ath_neg = 2.0

qgpv_mag_pos = 4e-5
theta_mag_pos = 0.
az_pos = 2.5
ax_pos = 1000.
ay_pos = 500.
#ath_pos = 2.0

qgpvlow_mag_neg = 0.0 #-5.0e-5
thetalow_mag_neg = 0.
azlow_neg = 1.0
axlow_neg = 200.
aylow_neg = 200.

qgpvlow_mag_pos = 1.25e-4
thetalow_mag_pos = 0.
azlow_pos = 1.0
axlow_pos = 600.
aylow_pos = 200.

external_qg_path = '/p/work1/lloveras/updated_adjoint/in_files/qgpv_adj.npy'
external_z_path = '/p/work1/lloveras/updated_adjoint/in_files/z_adj.npy'

## 2) WRF-specific parameters

In [ ]:
### Input file
file_name = '/p/work/lloveras/bwave/models/WRFV3_20km_old/run/wrfinput_d01'
netcdf_type = 'NETCDF3_64BIT_OFFSET'
title_str = "OUTPUT FROM IDEAL V3.6.1 PREPROCESSOR"  # title of netCDF file
time_str = "2021-01-01_00:00:00"  # start time of simulation

### Namelist variables that appear in wrfinput_d01 file
dt = 120.
mp_physics = 17
ra_lw_physics = 0
ra_sw_physics = 0
sf_sfclay_physics = 1
sf_surface_physics = 0
bl_pbl_physics = 1
cu_physics = 1
num_soil_layers = 5
diff_opt = 1
km_opt = 4
damp_opt = 3
dampcoef = 0.4
khdif = 0.
kvdif = 0.
spec_bdy_width = 8
sf_lake_physics = 0
surface_input_source = 1
hypsometric_opt = 0
num_land_cat = 24
num_soil_cat = 16

## 3) Initial jet

In [ ]:
### Set up grids
ny_h = ny
ly = ny_h*hres*1000
dy_l, ylvls_l, dy_h, ylvls_h = epv_jet.y_grid(ly, ny_l, ny_h)
dpi_l, pilvls_l, dpi_h, pilvls_h, p_pi_l, p_pi_h = epv_jet.pi_grid(pbot, pibot, pitop, npi_l, npi_h)
znw, znu = epv_jet.eta_grid(nz)

### Low-resolution run
f_l = np.zeros((npi_l,ny_l))
pv_l, f_l, u_l, theta_l, pv_out_l = epv_jet.solve_PV_inversion(ly, ny_l, \
    dy_l, dytr, pim, dpitr, pvt, pvs, dpipv, npi_l, pibot, pitop, dpi_l, \
    dyth, thtop, dth, f_l, om, nit)

### Interpolate onto high-resolution grid
f_temp = np.zeros((npi_h,ny_l))
for j in range(ny_l):
    f_temp[:,j] = epv_jet.interp_decreasing(pilvls_l,pilvls_h,f_l[:,j],npi_l,npi_h)

f_interp = np.zeros((npi_h,ny_h))
for k in range(npi_h):
    f_interp[k,:] = epv_jet.interp_increasing(ylvls_l,ylvls_h,f_temp[k,:],ny_l,ny_h)

### High-resolution run
pv_h, f_h, u_h, theta_h, pv_out_h = epv_jet.solve_PV_inversion(ly, ny_h, \
    dy_h, dytr, pim, dpitr, pvt, pvs, dpipv, npi_h, pibot, pitop, dpi_h, \
    dyth, thtop, dth, f_interp, om, nit)

### Compute on eta grid
p_jet, theta_jet, u_jet, rho_jet, z_jet, dtdz_jet, n_jet = \
    epv_jet.eta_calc(ny_h, nz, pbot, ptop, znu, npi_h, p_pi_h, u_h, theta_h, f_h)

### Surface pressure
p_surf_jet = np.ones(ny)
p_surf_jet = p_surf_jet*pbot
if psurf_gradient:
    u_jet, rho_jet, p_jet, p_surf_jet, z_jet = epv_jet.faster_jet(nz, ny, u_jet,\
                            theta_jet, p_jet, p_mid, c, hres, ptop, z_jet, znu)

if shear:
    u_jet, rho_jet, p_jet, p_surf_jet, z_jet = epv_jet.baro_shear_sin(nz, ny, u_jet,\
                            theta_jet, p_jet, p_mid, c_shear_sin, hres, ptop, z_jet, znu, decay_shear)
    
if baroclinic:
    u_jet, rho_jet, p_jet, p_surf_jet, z_jet = epv_jet.baroclinic_shear(nz, ny, u_jet,\
                            theta_jet, p_jet, p_mid, c_shear_sin, hres, ptop, z_jet, znu)
    
### PV check
pv_jet = epv_jet.pv_check(nz, ny, u_jet, theta_jet, z_jet, hres*1000., rho_jet)*1e6

## 4) QGPV anomaly

In [ ]:
### Set up grids
xl, yl, x, y, xg, yg, dz, zp, facz = qgpv_pert.cartesian_mesh(nx, ny, nz, hres, zl)
kmax, lmax, facx, facy, dxg, dyg = qgpv_pert.spectral_mesh(nx, ny, xl, yl)

n_pert = n_jet[:,y_pert]
ns_pert = n_jet[:,ys_pert]
nsp_pert = n_jet[:,ysp_pert]
n_pert_pos = n_jet[:,y_pert_pos]
n_pert_neg = n_jet[:,y_pert_neg]
nlow_pert_pos = n_jet[:,ylow_pert_pos]
nlow_pert_neg = n_jet[:,ylow_pert_neg]

dtdz_pert = dtdz_jet[:,y_pert]
dtdzs_pert = dtdz_jet[:,ys_pert]
dtdzsp_pert = dtdz_jet[:,ysp_pert]
dtdz_pert_pos = dtdz_jet[:,y_pert_pos]
dtdz_pert_neg = dtdz_jet[:,y_pert_neg]
dtdzlow_pert_pos = dtdz_jet[:,ylow_pert_pos]
dtdzlow_pert_neg = dtdz_jet[:,ylow_pert_neg]

if pert_type==1:
    pvxy, ubcxy, lbcxy, pvsp, ubcsp, lbcsp, bu_fac = qgpv_pert.qgpv_anom_gauss(qgpv_mag,\
        theta_mag, x_pert, y_pert, z_pert, az, ax, ay, x, y, zp, xg, yg, nx, ny, \
        nz, n_pert, facz, dz, dtdz_pert)
elif pert_type==2:
    pvxy, ubcxy, lbcxy, pvsp, ubcsp, lbcsp, bu_fac = qgpv_pert.qgpv_anom_cos(qgpv_mag,\
        theta_mag, x_pert, y_pert, z_pert, az, ax, ay, x, y, zp, xg, yg, nx, ny, \
        nz, n_pert, facz, dz, dtdz_pert)    
elif pert_type==3:
    pvxy, ubcxy, lbcxy, pvsp, ubcsp, lbcsp, bu_fac = qgpv_pert.qgpv_theta(qgpv_mag,\
        theta_mag, x_pert, y_pert, th_pert, ath, ax, ay, x, y, zp, xg, yg, nx, ny, \
        nz, n_pert, facz, dz, theta_jet, z_jet, dtdz_pert)
    
### Invert anomaly
fbsp, ftsp, fzbsp, fztsp, fsp = qgpv_pert.qgpv_inversion(nx, ny, nz, bu_fac, \
        facx, facy, facz, kmax, lmax, pvsp, ubcsp, lbcsp, dz)

### Compute u, v, rho, theta perturbation fields
u_pert, v_pert, theta_pert, rho_pert, fxy = qgpv_pert.qgpv_solver(fsp, nx, ny, nz, \
        bu_fac, dxg, dyg, dz, lbcxy, ubcxy, dtdz_pert)

p_pert = fxy*F0

if up_pert == False:
    u_pert = np.zeros(np.shape(fxy))
    v_pert = np.zeros(np.shape(fxy))
    theta_pert = np.zeros(np.shape(fxy))
    rho_pert = np.zeros(np.shape(fxy))
    p_pert = np.zeros(np.shape(fxy))

if surf_pert:
    pvs, ubcs, lbcs, pvsps, ubcsps, lbcsps, bu_facs = qgpv_pert.qgpv_surf(dqgpv_surf, dtheta_surf,\
        xs_pert, ys_pert, azs, axs, ayn, ays, x, y, zp, xg, yg, nx, ny, nz, ns_pert, facz, dz, dtdzs_pert)
    fbsps, ftsps, fzbsps, fztsps, fsps = qgpv_pert.qgpv_inversion(nx, ny, nz, bu_facs, \
        facx, facy, facz, kmax, lmax, pvsps, ubcsps, lbcsps, dz)
    us_pert, vs_pert, thetas_pert, rhos_pert, fxys = qgpv_pert.qgpv_solver(fsps, nx, ny, nz, \
        bu_facs, dxg, dyg, dz, lbcs, ubcs, dtdzs_pert)

    ps_pert = fxys*F0
    
    u_pert = u_pert + us_pert
    v_pert = v_pert + vs_pert
    theta_pert = theta_pert + thetas_pert
    rho_pert = rho_pert + rhos_pert
    p_pert = p_pert + ps_pert
    
if extra_surfpert:
    pvsp, ubcsp, lbcsp, pvspsp, ubcspsp, lbcspsp, bu_facsp = qgpv_pert.qgpv_surf(dqgpv_pert, dtheta_pert,\
        xsp_pert, ysp_pert, azsp, axsp, aynp, aysp, x, y, zp, xg, yg, nx, ny, nz, nsp_pert, facz, dz, dtdzsp_pert)
    fbspsp, ftspsp, fzbspsp, fztspsp, fspsp = qgpv_pert.qgpv_inversion(nx, ny, nz, bu_facsp, \
        facx, facy, facz, kmax, lmax, pvspsp, ubcspsp, lbcspsp, dz)
    usp_pert, vsp_pert, thetasp_pert, rhosp_pert, fxysp = qgpv_pert.qgpv_solver(fspsp, nx, ny, nz, \
        bu_facsp, dxg, dyg, dz, lbcsp, ubcsp, dtdzsp_pert)

    psp_pert = fxysp*F0
    
    u_pert = u_pert + usp_pert
    v_pert = v_pert + vsp_pert
    theta_pert = theta_pert + thetasp_pert
    rho_pert = rho_pert + rhosp_pert
    p_pert = p_pert + psp_pert
    
if extra_upperts:
    ### Positive
    pvxy_pos, ubcxy_pos, lbcxy_pos, pvsp_pos, ubcsp_pos, lbcsp_pos, bu_fac_pos = qgpv_pert.qgpv_anom_cos(qgpv_mag_pos,\
        theta_mag_pos, x_pert_pos, y_pert_pos, z_pert_pos, az_pos, ax_pos, ay_pos, x, y, zp, xg, yg, nx, ny, \
        nz, n_pert_pos, facz, dz, dtdz_pert_pos)
    
    fbsp_pos, ftsp_pos, fzbsp_pos, fztsp_pos, fsp_pos = qgpv_pert.qgpv_inversion(nx, ny, nz, bu_fac_pos, \
        facx, facy, facz, kmax, lmax, pvsp_pos, ubcsp_pos, lbcsp_pos, dz)

    u_pert_pos, v_pert_pos, theta_pert_pos, rho_pert_pos, fxy_pos = qgpv_pert.qgpv_solver(fsp_pos, nx, ny, nz, \
        bu_fac_pos, dxg, dyg, dz, lbcxy_pos, ubcxy_pos, dtdz_pert_pos)
    
    ### Negative
    pvxy_neg, ubcxy_neg, lbcxy_neg, pvsp_neg, ubcsp_neg, lbcsp_neg, bu_fac_neg = qgpv_pert.qgpv_anom_cos(qgpv_mag_neg,\
        theta_mag_neg, x_pert_neg, y_pert_neg, z_pert_neg, az_neg, ax_neg, ay_neg, x, y, zp, xg, yg, nx, ny, \
        nz, n_pert_neg, facz, dz, dtdz_pert_neg)
    
    fbsp_neg, ftsp_neg, fzbsp_neg, fztsp_neg, fsp_neg = qgpv_pert.qgpv_inversion(nx, ny, nz, bu_fac_neg, \
        facx, facy, facz, kmax, lmax, pvsp_neg, ubcsp_neg, lbcsp_neg, dz)

    u_pert_neg, v_pert_neg, theta_pert_neg, rho_pert_neg, fxy_neg = qgpv_pert.qgpv_solver(fsp_neg, nx, ny, nz, \
        bu_fac_neg, dxg, dyg, dz, lbcxy_neg, ubcxy_neg, dtdz_pert_neg)
    
    p_pert_pos = fxy_pos*F0
    p_pert_neg = fxy_neg*F0
    
    u_pert = u_pert + u_pert_pos + u_pert_neg
    v_pert = v_pert + v_pert_pos + v_pert_neg
    theta_pert = theta_pert + theta_pert_pos + theta_pert_neg
    rho_pert = rho_pert + rho_pert_pos + rho_pert_neg
    p_pert = p_pert + p_pert_pos + p_pert_neg
    
if extra_lowperts:
    ### Positive
    pvxylow_pos, ubcxylow_pos, lbcxylow_pos, pvsplow_pos, ubcsplow_pos, lbcsplow_pos, bulow_fac_pos = qgpv_pert.qgpv_anom_cos(qgpvlow_mag_pos,\
        thetalow_mag_pos, xlow_pert_pos, ylow_pert_pos, zlow_pert_pos, azlow_pos, axlow_pos, aylow_pos, x, y, zp, xg, yg, nx, ny, \
        nz, nlow_pert_pos, facz, dz, dtdzlow_pert_pos)  
    
    fbsplow_pos, ftsplow_pos, fzbsplow_pos, fztsplow_pos, fsplow_pos = qgpv_pert.qgpv_inversion(nx, ny, nz, bulow_fac_pos, \
        facx, facy, facz, kmax, lmax, pvsplow_pos, ubcsplow_pos, lbcsplow_pos, dz)

    ulow_pert_pos, vlow_pert_pos, thetalow_pert_pos, rholow_pert_pos, fxylow_pos = qgpv_pert.qgpv_solver(fsplow_pos, nx, ny, nz, \
        bulow_fac_pos, dxg, dyg, dz, lbcxylow_pos, ubcxylow_pos, dtdzlow_pert_pos)
    
    ### Negative
    pvxylow_neg, ubcxylow_neg, lbcxylow_neg, pvsplow_neg, ubcsplow_neg, lbcsplow_neg, bulow_fac_neg = qgpv_pert.qgpv_anom_cos(qgpvlow_mag_neg,\
        thetalow_mag_neg, xlow_pert_neg, ylow_pert_neg, zlow_pert_neg, azlow_neg, axlow_neg, aylow_neg, x, y, zp, xg, yg, nx, ny, \
        nz, nlow_pert_neg, facz, dz, dtdzlow_pert_neg)  
    
    fbsplow_neg, ftsplow_neg, fzbsplow_neg, fztsplow_neg, fsplow_neg = qgpv_pert.qgpv_inversion(nx, ny, nz, bulow_fac_neg, \
        facx, facy, facz, kmax, lmax, pvsplow_neg, ubcsplow_neg, lbcsplow_neg, dz)

    ulow_pert_neg, vlow_pert_neg, thetalow_pert_neg, rholow_pert_neg, fxylow_neg = qgpv_pert.qgpv_solver(fsplow_neg, nx, ny, nz, \
        bulow_fac_neg, dxg, dyg, dz, lbcxylow_neg, ubcxylow_neg, dtdzlow_pert_neg)
    
    plow_pert_pos = fxylow_pos*F0
    plow_pert_neg = fxylow_neg*F0
    
    u_pert = u_pert + ulow_pert_pos + ulow_pert_neg
    v_pert = v_pert + vlow_pert_pos + vlow_pert_neg
    theta_pert = theta_pert + thetalow_pert_pos + thetalow_pert_neg
    rho_pert = rho_pert + rholow_pert_pos + rholow_pert_neg
    p_pert = p_pert + plow_pert_pos + plow_pert_neg
    
if external_qg_pert:
    pvxy_adj_in = np.load(external_qg_path)
    z_adj_in = np.load(external_z_path)
    
    pvxy_adj = np.zeros((nz,ny,nx))
    for k in range(nz):
        pvxy_adj[k,:,:] = interplevel(pvxy_adj_in, z_adj_in, zp[k])
        
    pvxy_adj = np.nan_to_num(pvxy_adj)
    
    pvsp_adj = np.zeros((nz,ny,nx),dtype=np.dtype(np.complex64))
    for k in range(0,nz):
        pvsp_adj[k,:,:] = np.fft.fft2(np.squeeze(pvxy_adj[k,:,:]))
    
    fbsp_adj, ftsp_adj, fzbsp_adj, fztsp_adj, fsp_adj = qgpv_pert.qgpv_inversion(nx, ny, nz, bu_fac, \
            facx, facy, facz, kmax, lmax, pvsp_adj, ubcsp, lbcsp, dz)

    u_pert_adj, v_pert_adj, theta_pert_adj, rho_pert_adj, fxy_adj = qgpv_pert.qgpv_solver(fsp_adj, nx, ny, nz, \
            bu_fac, dxg, dyg, dz, lbcxy, ubcxy, dtdz_pert)
    
    p_pert_adj = fxy_adj*F0
    
    u_pert = u_pert + u_pert_adj
    v_pert = v_pert + v_pert_adj
    theta_pert = theta_pert + theta_pert_adj
    rho_pert = rho_pert + rho_pert_adj
    p_pert = p_pert + p_pert_adj

## 6) WRF input file

In [ ]:
### Set up 
dnw, rdnw, dn, rdn, fnp, fnm, cof1, cof2, cf1, cf2, cf3, cfn, cfn1, rdx,\
     rdy = wrf_fields.wrf_grid(nx, ny, nz, hres, znw)

### Add QGPV perturbation
u_eta, v_eta, theta_eta, rho_eta, pres_eta, z_eta, up_eta, vp_eta, \
    thetap_eta, rhop_eta, presp_eta = wrf_fields.add_pert(u_jet, theta_jet, \
    rho_jet, z_jet, p_jet, zp, u_pert, v_pert, rho_pert, theta_pert, p_pert, nx, ny, nz)

#p_bot = np.ones((ny,nx))*pbot + p_pert[0,:,:]
p_surf_eta = np.zeros((ny,nx))
for i in range(nx):
    p_surf_eta[:,i] = p_surf_jet

p_bot = p_surf_eta + p_pert[0,:,:]
p_top = np.ones((ny,nx))*ptop + p_pert[-1,:,:]

### Compute base fields in middle of domain
mub, pb, t_init, alb, phb = wrf_fields.middle_sounding(nx, ny, nz, p_bot, \
    p_top, znu, dnw, z_eta, theta_eta, rho_eta, u_eta, v_eta, pres_eta, avg_rh)             
        
### Compute fields in entire domain
u, v, t, ph, mu, p, moist, t_base, qv_base, u_base, v_base, \
    tsk, tmn = wrf_fields.full_domain(nx, ny, nz, p_bot, p_top, znu, \
                dnw, mub, pb, alb, dn, z_eta, theta_eta, rho_eta, u_eta, v_eta, pres_eta, avg_rh)     

In [ ]:
hres_m = hres*1000.
ncfile = nc.Dataset(file_name,'w',format=netcdf_type)
ncfile = write_wrfinputd01.write(ncfile, nx, ny, nz, hres_m, title_str, \
        time_str, u, v, t, ph, phb, t_init, mu, mub, p, pb, \
        fnm, fnp, rdnw, rdn, dnw, dn, t_base, cfn, cfn1, rdx, rdy, cf1, \
        cf2, cf3, tsk, u_base, v_base, qv_base, tmn, moist, znw, znu, \
        diff_opt, km_opt, damp_opt, dampcoef, khdif, kvdif, mp_physics, \
        ra_lw_physics, ra_sw_physics, sf_sfclay_physics, sf_surface_physics, \
        bl_pbl_physics, cu_physics, sf_lake_physics, surface_input_source, \
        hypsometric_opt, dt, num_land_cat, num_soil_layers, num_soil_cat, \
        spec_bdy_width, ptop)
ncfile.close()